# Laboratorio 6
## Universidad del Valle de Guatemala <br> Facultad de Ingeniería
#### Departamento de Ciencias de la Computación <br> Data Science - Sección 10
#### Cristian Laynez

## Problema 1 - Tráfico en la cuidad de Guatemala

In [2]:
# Importar librerias
import pandas as pd
import numpy as np
import joblib as jl
import nltk
import re
import pickle
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
import json
from datetime import datetime, timezone

In [3]:
from nltk import bigrams
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cristian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/cristian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/cristian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
stop_words = set(stopwords.words('spanish'))

#### 2. Cargar el archivo de datos.

In [9]:
data_frame = pd.read_csv('./data/traficogt.csv', encoding='utf-8')
data_frame

,Unnamed: 0,id,id_str,url,date,user,lang,rawContent,replyCount,retweetCount,...,quotedTweet,place,coordinates,inReplyToTweetId,inReplyToUser,source,sourceUrl,sourceLabel,media,_type
0,0,1701654244858679742,1701654244858679742,https://twitter.com/EmisorasUnidas/status/1701...,2023-09-12 17:49:21+00:00,"{'id': 40256008, 'id_str': '40256008', 'url': ...",es,"#AHORA Amílcar Montejo, director de Comunicaci...",1,2,...,"{'id': 1701651855212691764, 'id_str': '1701651...",NaN,NaN,NaN,NaN,"<a href=""https://twitter.com"" rel=""nofollow"">T...",https://twitter.com,TweetDeck Web App,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.Tweet
1,1,1701651855212691764,1701651855212691764,https://twitter.com/amilcarmontejo/status/1701...,2023-09-12 17:39:52+00:00,"{'id': 372126670, 'id_str': '372126670', 'url'...",es,CONDUCTORA QUE SE NEGÓ A MOVILIZAR VEHÍCULO ES...,149,78,...,NaN,NaN,NaN,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,{'photos': [{'url': 'https://pbs.twimg.com/med...,snscrape.modules.twitter.Tweet
2,2,1701348453916311903,1701348453916311903,https://twitter.com/edgarduarteagui/status/170...,2023-09-11 21:34:15+00:00,"{'id': 100815432, 'id_str': '100815432', 'url'...",es,Camión de arena volteado en el viaducto el pul...,2,1,...,NaN,NaN,NaN,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.Tweet
3,3,1701995859229958189,1701995859229958189,https://twitter.com/DrDavidCabrera/status/1701...,2023-09-13 16:26:49+00:00,"{'id': 743252657083650048, 'id_str': '74325265...",es,"Paciente de 39 años, dolor lumbar de 1 año tra...",3,17,...,NaN,NaN,NaN,NaN,NaN,"<a href=""https://help.twitter.com/en/using-twi...",https://help.twitter.com/en/using-twitter/how-...,advertiser-interface,{'photos': [{'url': 'https://pbs.twimg.com/med...,snscrape.modules.twitter.Tweet
4,4,1701216420997017888,1701216420997017888,https://twitter.com/EmisorasUnidas/status/1701...,2023-09-11 12:49:36+00:00,"{'id': 40256008, 'id_str': '40256008', 'url': ...",es,"#AHORA Amílcar Montejo, director de Comunicaci...",0,0,...,NaN,NaN,NaN,NaN,NaN,"<a href=""https://twitter.com"" rel=""nofollow"">T...",https://twitter.com,TweetDeck Web App,{'photos': [{'url': 'https://pbs.twimg.com/med...,snscrape.modules.twitter.Tweet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12626,12626,1237533600368562176,1237533600368562176,https://twitter.com/SoyGtm/status/123753360036...,2020-03-11 00:19:37+00:00,"{'id': 3729523453, 'id_str': '3729523453', 'ur...",es,Alguien sabe que paso en la 7av z9. Llevo 45 m...,0,0,...,NaN,"{'id': '13d479b108707983', 'fullName': 'Guatem...",NaN,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,{'photos': [{'url': 'https://pbs.twimg.com/med...,snscrape.modules.twitter.Tweet
12627,12627,1237513419122593794,1237513419122593794,https://twitter.com/arturocampos61/status/1237...,2020-03-10 22:59:25+00:00,"{'id': 144944157, 'id_str': '144944157', 'url'...",es,Pregunta sería para @amilcarmontejo los inefic...,0,0,...,NaN,NaN,NaN,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.Tweet
12628,12628,1237443272445960197,1237443272445960197,https://twitter.com/C_J_El_Lobo/status/1237443...,2020-03-10 18:20:41+00:00,"{'id': 1852058552, 'id_str': '1852058552', 'ur...",es,fuerte congestionamiento en diagonal 6 provoca...,0,0,...,NaN,NaN,NaN,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,{'photos': [{'url': 'https://pbs.twimg.com/med...,snscrape.modules.twitter.Tweet
12629,12629,1237356194957754368,1237356194957754368,https://twitter.com/ElCan

In [10]:
# Ver todas las columnas
data_frame.columns.tolist()

['Unnamed: 0',
 'id',
 'id_str',
 'url',
 'date',
 'user',
 'lang',
 'rawContent',
 'replyCount',
 'retweetCount',
 'likeCount',
 'quoteCount',
 'conversationId',
 'hashtags',
 'cashtags',
 'mentionedUsers',
 'links',
 'viewCount',
 'retweetedTweet',
 'quotedTweet',
 'place',
 'coordinates',
 'inReplyToTweetId',
 'inReplyToUser',
 'source',
 'sourceUrl',
 'sourceLabel',
 'media',
 '_type']

#### 3. Limpie y preprocese los datos (Describir de forma detallada las actividades de preprocesamiento que se llevó a cabo)

In [11]:
# Eliminar las siguientes columnas
# Unnamed: 0, url, id_str, coordinates, inReplyToTweetId, inReplyToUser, media, _type
columns_to_delete = [
    "Unnamed: 0", "url", "id_str", "coordinates", 
    "inReplyToTweetId", "inReplyToUser", "media",
    "_type", "conversationId", "cashtags", "links",
    "retweetedTweet"
]

for column in columns_to_delete:
    data_frame.drop(column, inplace=True, axis=1)

In [35]:
list_usernames = []
list_raw_descriptions = []
# username, displayname, rawDescription
for user_value in data_frame["user"]:
    username_pattern = r"'username': '([^']+)'"
    raw_description_pattern = r"'rawDescription': '([^']+)'"

    usernames = re.findall(username_pattern, user_value)
    raw_descriptions = re.findall(raw_description_pattern, user_value)
    
    list_usernames.append(usernames[0])
    list_raw_descriptions.append("-1" if len(raw_descriptions) == 0 else raw_descriptions[0])

In [27]:
data_frame["username"] = list_usernames
data_frame["rawDescription"] = list_raw_descriptions

In [28]:
data_frame

,id,date,user,lang,rawContent,replyCount,retweetCount,likeCount,quoteCount,hashtags,mentionedUsers,viewCount,quotedTweet,place,source,sourceUrl,sourceLabel,username,rawDescription
0,1701654244858679742,2023-09-12 17:49:21+00:00,"{'id': 40256008, 'id_str': '40256008', 'url': ...",es,ahora amílcar montejo director comunicación em...,1,2,8,0,"['AHORA', 'TráficoGT']","[{'id': 372126670, 'username': 'amilcarmontejo...",6645.0,"{'id': 1701651855212691764, 'id_str': '1701651...",NaN,"<a href=""https://twitter.com"" rel=""nofollow"">T...",https://twitter.com,TweetDeck Web App,EmisorasUnidas,'
1,1701651855212691764,2023-09-12 17:39:52+00:00,"{'id': 372126670, 'id_str': '372126670', 'url'...",es,conductora negó movilizar vehículo multada cal...,149,78,524,49,[],[],163347.0,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,amilcarmontejo,'
2,1701348453916311903,2023-09-11 21:34:15+00:00,"{'id': 100815432, 'id_str': '100815432', 'url'...",es,camión arena volteado viaducto pulté hacia hac...,2,1,4,0,"['transitogt', 'traficogt']","[{'id': 372126670, 'username': 'amilcarmontejo...",1033.0,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,edgarduarteagui,'
3,1701995859229958189,2023-09-13 16:26:49+00:00,"{'id': 743252657083650048, 'id_str': '74325265...",es,pacient 39 año dolor lumbar 1 año tra caída hi...,3,17,95,0,[],[],49375.0,NaN,NaN,"<a href=""https://help.twitter.com/en/using-twi...",https://help.twitter.com/en/using-twitter/how-...,advertiser-interface,DrDavidCabrera,'
4,1701216420997017888,2023-09-11 12:49:36+00:00,"{'id': 40256008, 'id_str': '40256008', 'url': ...",es,ahora amílcar montejo director comunicación em...,0,0,20,0,"['AHORA', 'TráficoGT']","[{'id': 372126670, 'username': 'amilcarmontejo...",4823.0,NaN,NaN,"<a href=""https://twitter.com"" rel=""nofollow"">T...",https://twitter.com,TweetDeck Web App,EmisorasUnidas,'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12626,1237533600368562176,2020-03-11 00:19:37+00:00,"{'id': 3729523453, 'id_str': '3729523453', 'ur...",es,alguien sabe paso 7av z9 llevo 45 minuto parad...,0,0,0,0,"['TraficoGT', 'TransitoGT']","[{'id': 372126670, 'username': 'amilcarmontejo...",NaN,NaN,"{'id': '13d479b108707983', 'fullName': 'Guatem...","<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,SoyGtm,'
12627,1237513419122593794,2020-03-10 22:59:25+00:00,"{'id': 144944157, 'id_str': '144944157', 'url'...",es,pregunta amilcarmontejo ineficient agent manej...,0,0,3,0,['TraficoGT'],"[{'id': 372126670, 'username': 'amilcarmontejo...",NaN,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,arturocampos61,'
12628,1237443272445960197,2020-03-10 18:20:41+00:00,"{'id': 1852058552, 'id_str': '1852058552', 'ur...",es,fuert congestionamiento diagon 6 provocado cam...,0,0,9,0,['TraficoGT'],"[{'id': 372126670, 'username': 'amilcarmontejo...",NaN,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,C_J_El_Lobo,'
12629,1237356194957754368,2020-03-10 12:34:40+00:00,"{'id': 603040343, 'id_str': '603040343', 'url'...",es,don amilcarmontejo motorista aquí calzada paz ...,3,2,13,0,['traficogt'],"[{'id': 372126670, 'username': 'amilcarmontejo...",NaN,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,ElCancheArriaza,'


In [14]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

In [15]:
def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [16]:
def get_stemmed_text(text):    
    stemmer = PorterStemmer()
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

In [17]:
def apply_lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

In [18]:
# Convertir el texto en minúsculas
data_frame['rawContent'] = data_frame['rawContent'].str.lower()

# Quitar caracteres especiales “#”,”@” o los apóstrofes
data_frame['rawContent'] = data_frame['rawContent'].str.replace(r'[#@\'"]', '', regex=True)

# Quitar urls
data_frame['rawContent'] = data_frame['rawContent'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', regex=True)

# Eliminar signos de puntuacion
data_frame['rawContent'] = data_frame['rawContent'].str.replace(r'[^\w\s]', '', regex=True)

In [19]:
# Revisar si hay emojis en el texto y luego quitarlos
data_frame['rawContent'] = data_frame['rawContent'].apply(remove_emojis)

In [20]:
# Quitar los artículos, preposiciones y conjunciones (stopwords)
data_frame['rawContent'] = data_frame['rawContent'].apply(remove_stopwords)

In [21]:
# Normalizar las palabras para tratar de convertir todas las palabras encontradas en solo una palabra
data_frame['rawContent'] = data_frame['rawContent'].apply(get_stemmed_text)

In [22]:
# Aplicar la función de lematización al atributo 'text' para identificar la parte
# del discurso de una palabra determinada y luego aplicando reglas más complejas 
# para transformar la palabra en su verdadera raíz
data_frame['rawContent'] = data_frame['rawContent'].apply(apply_lemmatization)